In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import os; import re

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
from sklearn.cross_validation import KFold

In [6]:
from sklearn.grid_search import GridSearchCV

In [7]:
import time
import datetime

In [8]:
features = pd.read_csv('./_features/features.csv', index_col='match_id')

In [36]:
dict_names = os.listdir("./_data/data/dictionaries")

###### create dict of dicts with object description

In [203]:
dicts = dict()
for i,d in enumerate(dict_names):
    dicts[re.sub(r'(.*)\.csv',r'\1',d)] = pd.read_csv('./_data/data/dictionaries/%s' % d)
#print dicts

###### drop final attributes

In [9]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [58]:
counts=X.count()

In [82]:
skips=counts[counts.values < 97230].keys()

###### не в каждом матче FB происходит в первые 5 минут, не в каждом матче летучий курьер приобретается одной из команд в первые 5 минут

In [87]:
skips[[0,-2]]

Index([u'first_blood_time', u'dire_flying_courier_time'], dtype='object')

###### replace na items to ZERO

In [10]:
X_na = X.fillna(0)

###### target var - radiant_win

In [11]:
y=features.radiant_win

###### create cross validator

In [12]:
cv = KFold(y.size,shuffle=True, n_folds=5)

###### сreate and fit GradientBoostingClassifier

In [193]:
times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [194]:
for n in [10,20,30,60]:
    params_grid={"n_estimators":[n]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    times.loc[n_est] = [n_est,et,sc]


#print 'Time elapsed:', datetime.datetime.now() - start_time
times

,n_estimators,time,score
10,10,63,0.66469
20,20,126,0.682274
30,30,184,0.689361
60,60,505,0.700428


###### set subsample to 0.2

In [195]:
op_times = pd.DataFrame(columns=['n_estimators','time','score'],index=[10,20,30,60])

In [202]:
for n in [10,20,30,60]:
    params_grid={"n_estimators":[n],'loss':['deviance'],'subsample':[0.1]}
    gs = GridSearchCV(GradientBoostingClassifier(),param_grid=params_grid,cv=cv,scoring="roc_auc")
    start_time = datetime.datetime.now()
    #times['n_estimators:' + gs.param_grid.values()[0][0]] = 
    gs.fit(X_na,y)
    n_est=gs.param_grid.values()[0][0]
    et=(datetime.datetime.now() - start_time).seconds
    sc=gs.best_score_
    
    op_times.loc[n_est] = [n_est,et,sc]

#print 'Time elapsed:', datetime.datetime.now() - start_time
op_times

,n_estimators,time,score
10,10,48,0.672941
20,20,100,0.685408
30,30,133,0.692446
60,60,297,0.702956


# Regression

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
from sklearn.metrics import roc_auc_score

###### prepare data

In [16]:
Xtest = pd.read_csv('./_features_test/features_test.csv', index_col='match_id')

###### drop final attributes

In [17]:
X = features.drop(features.columns[-6:],1)

###### found skips

In [18]:
counts=X.count()

In [19]:
skips=counts[counts.values < 97230].keys()

###### replace na items to ZERO

In [20]:
X_na = X.fillna(0)

###### target var - radiant_win

In [21]:
y=features.radiant_win

###### create crossval generator

In [22]:
cv = KFold(y.size,shuffle=True, n_folds=5)

###### drop categorial and hero features

In [23]:
keys=list(X.keys()[pd.Series(X.keys()).str.contains('hero', na=False)])

keys=keys+['lobby_type'] #,'start_time']+X.keys()[pd.Series(X.keys()).str.contains('first_blood_player', na=False)].tolist()

Xdr = X_na.drop(keys,1)

###### transform scale

In [24]:
scdr=StandardScaler()

Xsdr=scdr.fit_transform(Xdr)

###### dummy heroes features

In [25]:
N=112

In [26]:
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

###### concat scale and dummy hero features

In [27]:
XX=np.column_stack((Xsdr,X_pick))

###### create and fit estimator, with cross validation

###### качество на всех исходных признаках

In [374]:
params={'C':[0.01]}

In [376]:
grid_no_dummy = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [377]:
sc=StandardScaler()

Xs=sc.fit_transform(X_na)

In [378]:
grid_no_dummy.fit(Xs,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [379]:
grid_no_dummy.best_score_

0.71632892946425031

In [380]:
grid_no_dummy.best_params_

{'C': 0.040914285714285716}

###### качество после удаления категориальных признаков

In [381]:
grid_no_dummy_dr = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [382]:
grid_no_dummy_dr.fit(Xsdr,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [383]:
grid_no_dummy_dr.best_score_

0.71642814592328452

In [384]:
grid_no_dummy_dr.best_params_

{'C': 0.040914285714285716}

###### Качество с мешком слов

In [385]:
grid = GridSearchCV(LogisticRegression(),cv=cv,param_grid=params,n_jobs=-1,scoring='roc_auc')

In [386]:
grid.fit(XX,y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=97230, n_folds=5, shuffle=True, random_state=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-04,   4.09143e-02,   8.17286e-02,   1.22543e-01,
         1.63357e-01,   2.04171e-01,   2.44986e-01,   2.85800e-01,
         3.26614e-01,   3.67429e-01,   4.08243e-01,   4.49057e-01,
         4.89871e-01,   5.30686e-01,   5.71500e-01,   6.12314e-01,
         6.5312...    1.79593e+00,   1.83674e+00,   1.87756e+00,   1.91837e+00,
         1.95919e+00,   2.00000e+00])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [387]:
grid.best_score_

0.75172924222139459

In [388]:
grid.best_params_

{'C': 0.040914285714285716}

###### predict by test data

In [389]:
estL=LogisticRegression(C=0.040914285714285716)

In [390]:
estL.fit(XX,y)

LogisticRegression(C=0.0409142857143, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

###### transform test data

In [392]:
Xtest_na = Xtest.fillna(0)

In [393]:
Xtest_dr=Xtest_na.drop(keys,1)

In [394]:
tscdr=StandardScaler()

Xtsdr=tscdr.fit_transform(Xtest_dr)

In [192]:
Xt_pick = np.zeros((Xtest.shape[0], N))

for i, match_id in enumerate(Xtest.index):
    for p in xrange(5):
        Xt_pick[i, Xtest.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        Xt_pick[i, Xtest.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [395]:
XXt=np.column_stack((Xtsdr,Xt_pick))

###### predict 

In [396]:
pred=estL.predict_proba(XXt)[:, 1]

In [397]:
pred.max()

0.99637746303166252

In [398]:
pred.min()

0.0085410679923746558

In [399]:
pd.Series(pred).value_counts().max()

1

In [400]:
pred.mean()

0.51819197738485101